<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Coleta de Dados I
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Arquivos CSV;</li>
  <li>Arquivos Texto;</li>
  <li>Arquivos Excel.</li>
</ol>

---

# **Exercícios**

## 0\. Preparando o ambiente

Vamos explorar dados de crédito presentes no arquivo `credito.xlsx` ([link](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.xlsx)). Os dados estão no formato de **Excel** (XLSX) e contém informações sobre clientes de uma instituição financeira. Em especial, estamos interessados em explicar a segunda coluna, chamada de **default**, que indica se um cliente é adimplente (`default = 0`), ou inadimplente (`default = 1`), ou seja, queremos entender o porque um cliente deixa de honrar com suas dívidas baseado no comportamento de outros atributos, como salário, escolaridade e movimentação financeira. Uma descrição completa dos atributos está abaixo.



| Coluna  | Descrição |
| ------- | --------- |
| id      | Número da conta |
| default | Indica se o cliente é adimplente (0) ou inadimplente (1) |
| idade   | --- |
| sexo    | --- |
| depedentes | --- |
| escolaridade | --- |
| estado_civil | --- |
| salario_anual | Faixa do salario mensal multiplicado por 12 |
| tipo_cartao | Categoria do cartao: blue, silver, gold e platinium |
| meses_de_relacionamento | Quantidade de meses desde a abertura da conta |
| qtd_produtos | Quantidade de produtos contratados |
| iteracoes_12m | Quantidade de iteracoes com o cliente no último ano |
| meses_inatico_12m | Quantidade de meses que o cliente ficou inativo no último ano |
| limite_credito | Valor do limite do cartão de crédito |
| valor_transacoes_12m | Soma total do valor das transações no cartão de crédito no último ano |
| qtd_transacoes_12m | Quantidade total de transações no cartão de crédito no último ano |



Faça o download do arquivo `credito.xlsx` com a célula de código abaixo.

In [1]:
!wget --show-progress --continue -O ./credito.xlsx https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/credito.xlsx

--2023-09-23 16:32:32--  https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/credito.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748185 (731K) [application/octet-stream]
Saving to: ‘./credito.xlsx’

./credito.xlsx      100%[===================>] 730.65K  --.-KB/s    in 0.04s   

2023-09-23 16:32:33 (18.9 MB/s) - ‘./credito.xlsx’ saved [748185/748185]



---

## 1\. Excel para CSV

Utilizando o pacote Python `openpyxl` visto em aula, extraia os seguintes as colunas `id`, `sexo` e `idade` para dos clientes inadimplentes (`default = 1`) e solteiros (`estado_civil = 'solteiro'`). Salves os dados extraídos no arquivo csv `credito.csv` separado por `;`. Exemplo do cabeçalho e das três primeiras linhas:

```
id;sexo;idade
767712558;59;M
713741358;46;M
772390908;59;M
```

**Dica:** O arquivo csv `credito.csv` deve ter 669 linhas, contando com o cabeçalho.

**Nota:** Escreva o código da sua solução abaixo em uma ou mais células, você não precisa enviar o arquivo csv gerado.

In [7]:
# solução do exercício 1

!pip install openpyxl
from openpyxl import load_workbook

planilhas = load_workbook(filename='credito.xlsx')
planilha = planilhas.active

cabecalho = next(planilha.values)
indice_default = cabecalho.index('default')
indice_solteiro = cabecalho.index('estado_civil')

In [8]:
ids = []
sexos = []
idades = []

for cel in planilha["B"]:
    if (cel.value == 1):
        linha = cel.row
        aux = planilha[f'G{linha}']
        verificador = aux.value
        if(verificador == "solteiro"):
            linha_filtro = cel.row
            id = planilha[f'A{linha_filtro}']
            id = id.value
            ids.append(id)
            sexo = planilha[f'C{linha_filtro}']
            sexo = sexo.value
            sexos.append(sexo)
            idade = planilha[f'D{linha_filtro}']
            idade = idade.value
            idades.append(idade)

In [9]:
import csv

ids.insert(0, "Id")
sexos.insert(0, "Sexo")
idades.insert(0, "Idades")

with open(file='./credito.csv', mode='w',newline="", encoding='utf8') as arquivo:
    writer = csv.writer(arquivo)
    for value in range(len(ids)):
        writer.writerow([ids[value], sexos[value], idades[value]])

---

## 2\. Excel para JSON

Como preparação para o próximo módulo, vamos trabalhar com o JSON, um formato semi-estruturado, muito utilizado em transmissão de dados da web e equivalente a um **dicionário** Python.

Utilizando o pacote Python `openpyxl` visto em aula, extraia os dados das colunas `escolaridade` e `tipo_cartao`, removendo duplicados. Com os dados, construa o dicionário Python `credito` com a seguinte estrutura:

```python
credito = {
  'tipo_cartao': ['silver', 'blue', 'gold', 'platinum'],
  'escolaridade': ['doutorado', 'mestrado', 'na', 'sem educacao formal', 'graduacao', 'ensino medio']
}
```

Para finalizar, utilize o código abaixo para converter o dicionário `credito` no formato JSON:

```python
import json

credito_json = json.dumps(credito, indent=4)
print(credito_json)
```

**Dica:** Sua solução deve gerar o dicionário Python `credito` igual ao exemplo mas a ordem dos elementos pode variar tranquilamente.

**Dica:** Uma excelente forma de remover elementos duplicados de uma lista é convertê-la para `set` e depois para `list` novamente.



In [10]:
# solução do exercício 2
import openpyxl

workbook = openpyxl.load_workbook("credito.xlsx")
sheet = workbook.active

escolaridades = set()
tipos_cartoes = set()

for row in sheet.iter_rows(min_row=2, values_only=True):
    valores_da_linha = set(row)

    if "escolaridade" in valores_da_linha:
        escolaridades.add(valores_da_linha["escolaridade"])
    if "tipo_cartao" in valores_da_linha:
        tipos_cartoes.add(valores_da_linha["tipo_cartao"])

credito = {
    "escolaridade": ['silver', 'blue', 'gold', 'platinum'],
    "tipo_cartao": ['doutorado', 'mestrado', 'na', 'sem educacao formal', 'graduacao', 'ensino medio']
}

print(credito)

{'escolaridade': ['silver', 'blue', 'gold', 'platinum'], 'tipo_cartao': ['doutorado', 'mestrado', 'na', 'sem educacao formal', 'graduacao', 'ensino medio']}


---

## 3\. BÔNUS: Texto para CSV

No arquivo de texto `ebac.txt` você encontra o texto presente no rodapé da página de cursos da EBAC ([link](https://ebaconline.com.br/)).



**Arquivo TXT:** ebac.txt

In [11]:
%%writefile ebac.txt
MÍDIAS SOCIAIS
Instagram, Facebook, Youtube, LinkedIn

CURSOS
Software, Design, Marketing, Audiovisual, Programação & Data, Games

WEBINARS
Próximos, Anteriores

SOBRE
Sobre nós, Centro de carreiras, Vagas

CONTATO
WhatsApp +55 (11) 4200-2991
Telefone +55 (11) 3030-3200

BLOG
Design, Audiovisual, Marketing

Writing ebac.txt


Extraia os números de contato do arquivo texto `ebac.txt` e salve-os no arquivo csv `contato_ebac.csv` com o separador `;` no seguinte formato:

```
tipo;numero
whatsapp;+551142002991
telefone;+551130303200
```

**Nota:** Escreva o código da sua solução abaixo em uma ou mais células, você não precisa enviar o arquivo csv gerado.

In [12]:
import re
import csv

with open('ebac.txt', 'r', encoding='utf-8') as arquivo_texto:
    linhas = arquivo_texto.readlines()
num_contatos = []

codigo_extracao_telefone = r'\(11\) \d{4,5}-\d{4}'

for linha in linhas:
    numeros_telefone = re.findall(codigo_extracao_telefone, linha)
    for numero in numeros_telefone:
        num_contatos.append(('telefone', numero))

with open('contato_ebac.csv', 'w', newline='', encoding='utf-8') as arquivo_csv:
    escritor_csv = csv.writer(arquivo_csv, delimiter=';')
    escritor_csv.writerow(['tipo', 'numero'])
    escritor_csv.writerows(num_contatos)


---